In [183]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as op
import scipy.io

In [184]:
def sigmoid(z):
    result = 1.0 / (1.0 + np.exp(-z))
    
    return result

In [185]:
def sigmoidGradient(z):
    result = np.multiply(sigmoid(z), (1-sigmoid(z)))
    
    return result

In [186]:
def debugInitializeWeights(fan_out, fan_in):
    W = np.zeros((fan_out, fan_in+1))

    
    for i in range(fan_out):
        for j in range(fan_in+1):
            W[i][j] = np.sin(j*fan_out + i + 1) / 10
    
    
    return W

In [187]:
def computeNumericalGradient(theta, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t):
    
    m = theta.shape[0]
    numgrad = np.zeros(m)
    perturb = np.zeros(m)
    
    e = 1e-4
    
    
    for i in range(m):
        perturb[i] = e
        
        loss1 = computeCost(theta - perturb, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)
        loss2 = computeCost(theta + perturb, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)

        numgrad[i] = (loss2 - loss1) / (2*e)
        perturb[i] = 0        
    
    return numgrad

In [188]:
def checkNNGradients(lambda_t):
    input_layer_size = 3
    hidden_layer_size = 5
    num_labels = 3
    m = 5
    
    Theta1 = debugInitializeWeights(hidden_layer_size, input_layer_size)
    Theta2 = debugInitializeWeights(num_labels, hidden_layer_size)        
    
    X  = debugInitializeWeights(m, input_layer_size - 1)
    y = np.zeros((m, 1))
    for i in range(m):
        y[i][0] = 1 + (i+1) % num_labels
    y = y.astype(int)
    
    nn_params = np.concatenate((Theta1.flatten('F'), Theta2.flatten('F')))
    grad = computeGradient(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)
    
    numgrad = computeNumericalGradient(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)

    print(numgrad)
    print(grad)
    
    print("The above two columns you get should be very similar.")
    print("(Left-Your Numerical Gradient, Right-Analytical Gradient)")
    
    diff = np.linalg.norm(numgrad-grad)/np.linalg.norm(numgrad+grad)
    
    print('If your backpropagation implementation is correct, then',
          'the relative difference will be small (less than 1e-9). \n', 
          'Relative Difference: \n', diff)

In [189]:
def computeCost(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t):

    Theta1_flat = nn_params[ : hidden_layer_size * (input_layer_size + 1)]
    Theta2_flat = nn_params[hidden_layer_size * (input_layer_size + 1) : ]
    
    Theta1 = np.reshape(Theta1_flat, (hidden_layer_size, input_layer_size+1), 'F')
    Theta2 = np.reshape(Theta2_flat, (num_labels, hidden_layer_size+1), 'F')
    
    m = X.shape[0]

    a1 = np.c_[np.ones([m, 1]), X]    # Add a column of ones to x
    z2 = np.dot(a1, Theta1.T)    # 5000*25;

    a2 = sigmoid(z2)    # 5000*25;
    a2 = np.c_[np.ones([m, 1]), a2]     # 5000*26;

    z3 = np.dot(a2, Theta2.T)    # 5000*10;
    a3 = sigmoid(z3)    # 5000*10;           

    y_matrix = np.zeros([m, num_labels]); # 5000*10;              

    for i in range(m):
        if y[i][0]==0:
            y_matrix[i][9] = 1
        else:
            y_matrix[i][y[i][0]-1] = 1     


    sum_t = 0
    for i in range(m):
        for j in range(num_labels):
            sum_t = sum_t - y_matrix[i][j] * np.log(a3[i][j]) - (1-y_matrix[i][j]) * np.log(1-a3[i][j])

    J = sum_t / m

    sum_left = 0
    for i in range(Theta1.shape[0]):
        for j in range(1, Theta1.shape[1]):
            sum_left = sum_left + Theta1[i][j]**2

    sum_right = 0
    for i in range(Theta2.shape[0]):
        for j in range(1, Theta2.shape[1]):
            sum_right = sum_right + Theta2[i][j]**2

    J = J + (lambda_t/(2*m))*(sum_left + sum_right);
    
    return J

In [190]:
def computeGradient(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t):

    Theta1_flat = nn_params[ : hidden_layer_size * (input_layer_size + 1)]
    Theta2_flat = nn_params[hidden_layer_size * (input_layer_size + 1) : ]
    
    Theta1 = np.reshape(Theta1_flat, (hidden_layer_size, input_layer_size+1), 'F') # 25*401;
    Theta2 = np.reshape(Theta2_flat, (num_labels, hidden_layer_size+1), 'F') # 10*26;
    
    # nnCostFunction
    Theta1_grad = np.zeros(Theta1.shape) # 25*401;
    Theta2_grad = np.zeros(Theta2.shape) # 10*26;

    m = X.shape[0]
    
    a1 = np.c_[np.ones([m, 1]), X]    # Add a column of ones to x
    z2 = np.dot(a1, Theta1.T)    # 5000*25;

    a2 = sigmoid(z2)    # 5000*25;
    a2 = np.c_[np.ones([m, 1]), a2]     # 5000*26;

    z3 = np.dot(a2, Theta2.T)    # 5000*10;
    a3 = sigmoid(z3)    # 5000*10;           

    y_matrix = np.zeros([m, num_labels]); # 5000*10;              

    for i in range(m):
        if y[i][0]==0:
            y_matrix[i][9] = 1
        else:
            y_matrix[i][y[i][0]-1] = 1     

    
    ##-----------------------calculate Theta1_grad and Theta2_grad---------------------------
    for i in range(m):
        delta3 = a3[i] - y_matrix[i]
        delta3 = np.reshape(delta3, (1, -1)) # 1*10;
        
        term1 = np.dot(delta3, Theta2)
        term1 = (term1.T)[1:]
        term1 = np.reshape(term1, (1, -1))
        
        term2 = sigmoidGradient(z2[i])
        term2 = np.reshape(term2, (1, -1))
        
        delta2 = np.multiply(term1, term2) # 1*25;

        Theta1_grad = Theta1_grad + np.dot(delta2.T, np.reshape(a1[i], (1, -1)))    # 25*401;
        Theta2_grad = Theta2_grad + np.dot(delta3.T, np.reshape(a2[i], (1, -1)))    # 10*26;    

    Theta1_grad = Theta1_grad / m
    Theta2_grad = Theta2_grad / m
    
    Theta1_grad[:, 1:] += (lambda_t/m) * Theta1[:, 1:]
    Theta2_grad[:, 1:] += (lambda_t/m) * Theta2[:, 1:]
    
    
    grad = np.concatenate((Theta1_grad.flatten('F'), Theta2_grad.flatten('F')))
    
    return grad

In [191]:
def randInitializeWeights(L_in, L_out):
    epsilon_init = 0.12
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init
    
    return W

In [192]:
input_layer_size  = 400    # 20x20 Input Images of Digits
hidden_layer_size = 25    # 25 hidden units
num_labels = 10          # 10 labels, from 1 to 10   
                          # (note that we have mapped "0" to label 10)

## =========== Part 1: Loading and Visualizing Data =============
data = scipy.io.loadmat('ex4data1.mat')
X = data['X']
y = data['y']

y = np.reshape(y, (-1, 1))
y = y.astype(int)
m = X.shape[0]

## ================ Part 2: Loading Parameters ================
weights = scipy.io.loadmat('ex4weights.mat')
Theta1 = weights['Theta1'] # array
Theta2 = weights['Theta2']

print(X.shape)
print(y.shape)
print(Theta1.shape)
print(Theta2.shape)

(5000, 400)
(5000, 1)
(25, 401)
(10, 26)


In [193]:
## ================ Part 3: Compute Cost (Feedforward) ================
print('Feedforward Using Neural Network ...')

lambda_t = 0

nn_params = np.concatenate((Theta1.flatten('F'), Theta2.flatten('F')))

J = computeCost(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)
print('Cost at parameters (loaded from ex4weights): (this value should be about 0.287629)\n', J)

Feedforward Using Neural Network ...
Cost at parameters (loaded from ex4weights): (this value should be about 0.287629)
 0.28762916516132025


In [194]:
## =============== Part 4: Implement Regularization ===============

print('Checking Cost Function (w/ Regularization) ...')

lambda_t = 1

J = computeCost(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)

print('Cost at parameters (loaded from ex4weights): (this value should be about 0.383770)\n', J)

Checking Cost Function (w/ Regularization) ...
Cost at parameters (loaded from ex4weights): (this value should be about 0.383770)
 0.38376985909092476


In [195]:
## ================ Part 5: Sigmoid Gradient  ================
print('Evaluating sigmoid gradient...')
g = sigmoidGradient(np.array([-1, -0.5, 0, 0.5, 1]))
print('Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:')
print(g)

Evaluating sigmoid gradient...
Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [196]:
## ================ Part 6: Initializing Pameters ================
print('Initializing Neural Network Parameters ...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

initial_nn_params = np.concatenate((initial_Theta1.flatten('F'), initial_Theta2.flatten('F')))

print(initial_Theta1.shape)
print(initial_Theta2.shape)

Initializing Neural Network Parameters ...
(25, 401)
(10, 26)


In [197]:
## =============== Part 7: Implement Backpropagation ===============
print('Checking Backpropagation... ')
checkNNGradients(0)

Checking Backpropagation... 
[-9.27825235e-03  8.89911959e-03 -8.36010761e-03  7.62813551e-03
 -6.74798370e-03 -3.04978931e-06  1.42869427e-05 -2.59383093e-05
  3.69883235e-05 -4.68759764e-05 -1.75060084e-04  2.33146358e-04
 -2.87468731e-04  3.35320349e-04 -3.76215583e-04 -9.62660640e-05
  1.17982666e-04 -1.37149709e-04  1.53247082e-04 -1.66560292e-04
  3.14544970e-01  1.11056588e-01  9.74006970e-02  1.64090819e-01
  5.75736493e-02  5.04575855e-02  1.64567932e-01  5.77867378e-02
  5.07530173e-02  1.58339334e-01  5.59235296e-02  4.91620841e-02
  1.51127527e-01  5.36967009e-02  4.71456249e-02  1.49568335e-01
  5.31542052e-02  4.65597186e-02]
[-9.27825236e-03  8.89911960e-03 -8.36010762e-03  7.62813551e-03
 -6.74798370e-03 -3.04978914e-06  1.42869443e-05 -2.59383100e-05
  3.69883234e-05 -4.68759769e-05 -1.75060082e-04  2.33146357e-04
 -2.87468729e-04  3.35320347e-04 -3.76215587e-04 -9.62660620e-05
  1.17982666e-04 -1.37149706e-04  1.53247082e-04 -1.66560294e-04
  3.14544970e-01  1.1105658

In [198]:
## =============== Part 8: Implement Regularization ===============
print('Checking Backpropagation (w/ Regularization) ... ')
lambda_t = 3
checkNNGradients(lambda_t)

debug_J = computeCost(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)

print('Cost at (fixed) debugging parameters (w/ lambda =', lambda_t, '):', debug_J)
print('(for lambda = 3, this value should be about 0.576051)\n\n')

Checking Backpropagation (w/ Regularization) ... 
[-9.27825235e-03  8.89911959e-03 -8.36010761e-03  7.62813551e-03
 -6.74798370e-03 -1.67679797e-02  3.94334829e-02  5.93355565e-02
  2.47640974e-02 -3.26881426e-02 -6.01744725e-02 -3.19612287e-02
  2.49225535e-02  5.97717617e-02  3.86410548e-02 -1.73704651e-02
 -5.75658668e-02 -4.51963845e-02  9.14587966e-03  5.46101548e-02
  3.14544970e-01  1.11056588e-01  9.74006970e-02  1.18682669e-01
  3.81928666e-05  3.36926556e-02  2.03987128e-01  1.17148233e-01
  7.54801264e-02  1.25698067e-01 -4.07588279e-03  1.69677090e-02
  1.76337550e-01  1.13133142e-01  8.61628953e-02  1.32294136e-01
 -4.52964427e-03  1.50048382e-03]
[-9.27825236e-03  8.89911960e-03 -8.36010762e-03  7.62813551e-03
 -6.74798370e-03 -1.67679797e-02  3.94334829e-02  5.93355565e-02
  2.47640974e-02 -3.26881426e-02 -6.01744725e-02 -3.19612287e-02
  2.49225535e-02  5.97717617e-02  3.86410548e-02 -1.73704651e-02
 -5.75658668e-02 -4.51963845e-02  9.14587966e-03  5.46101547e-02
  3.14

In [199]:
## =================== Part 8: Training NN ===================
print('Training Neural Network...')    
lambda_t = 1    
#[nn_params, cost] = fmincg(costFunction, initial_nn_params, options);    
Result = op.minimize(fun = computeCost, 
                                 x0 = initial_nn_params, 
                                 args = (input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t),
                                 method = 'TNC',
                                 jac = computeGradient, 
                                 options={'maxiter':64})
nn_params_op = Result.x
cost = computeCost(nn_params_op, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)
print("cost: ", cost)


Theta1_flat_op = nn_params_op[ : hidden_layer_size * (input_layer_size + 1)]
Theta2_flat_op = nn_params_op[hidden_layer_size * (input_layer_size + 1) : ]
    
Theta1_op = np.reshape(Theta1_flat_op, (hidden_layer_size, input_layer_size+1), 'F') # 25*401;
Theta2_op = np.reshape(Theta2_flat_op, (num_labels, hidden_layer_size+1), 'F') # 10*26;

Training Neural Network...
cost:  0.7799570447574402


In [200]:
def predict(Theta1, Theta2, X):
    m = X.shape[0]
    num_labels = Theta2.shape[0]
    
    p = np.zeros((m, 1))
    h1 = sigmoid(np.dot(np.c_[np.ones([m, 1]), X], Theta1.T))
    h2 = sigmoid(np.dot(np.c_[np.ones([m, 1]), h1], Theta2.T)) # 5000 * 10;


    p = np.argmax(h2, axis=1) + 1
    p = np.reshape(p, (-1, 1))

    return p

In [201]:
## ================= Part 10: Implement Predict =================
pred = predict(Theta1_op, Theta2_op, X)
print('Training Set Accuracy: \n', np.sum(pred == y) * 100 / pred.shape[0])

Training Set Accuracy: 
 90.08
